In [74]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col

In [75]:
spark = SparkSession.builder.appName("Análise de Vendas").getOrCreate()

In [76]:
df = spark.read.option("delimiter", ",").option("quote", "'").option("header", True).csv("file:///mnt/notebooks/vendas.csv")

In [77]:
df.printSchema()
print(df.columns)

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- price: string (nullable = true)

['order_id', 'product_id', 'user_id', 'timestamp', 'quantity', 'price']


In [106]:
df = df.withColumn("quantity", col("quantity").cast("int"))
df = df.withColumn("price",    col("price").cast("double"))
df.show()

+--------+----------+-------+-------------------+--------+-----+
|order_id|product_id|user_id|          timestamp|quantity|price|
+--------+----------+-------+-------------------+--------+-----+
|       1|        A1|     U1|2025-05-06 10:00:00|       2| 10.0|
|       2|        B2|     U2|2025-05-06 10:15:00|       1| 25.0|
|       1|        C3|     U1|2025-05-06 10:30:00|       3|  5.0|
|       3|        A1|     U3|2025-05-06 10:45:00|       1| 10.0|
|       4|        D4|     U2|2025-05-06 11:00:00|       2| 15.0|
|       2|        B2|     U2|2025-05-06 11:15:00|       1| 25.0|
|       5|        C3|     U1|2025-05-06 11:30:00|       4|  5.0|
|       5|        E5|     U1|2025-05-06 11:45:00|       1| 50.0|
|       6|        A1|     U3|2025-05-06 12:00:00|       2| 10.0|
|       6|        D4|     U3|2025-05-06 12:15:00|       1| 15.0|
|       7|        F6|     U4|2025-05-06 12:30:00|       5|  8.0|
|       8|        G7|     U5|2025-05-06 12:45:00|       2| 30.0|
|       9|        A1|    

In [108]:
total_vendas = df.agg(
    F.sum(col("quantity") * col("price")).alias("total_vendas")
)
total_vendas.show()

+------------+
|total_vendas|
+------------+
|      1305.0|
+------------+



In [101]:
df.groupBy('product_id') \
  .agg(F.sum('quantity').alias('total_vendido')) \
  .orderBy(F.desc("total_vendido")) \
  .show(3)

+----------+-------------+
|product_id|total_vendido|
+----------+-------------+
|        C3|           19|
|        A1|           13|
|        B2|           11|
+----------+-------------+
only showing top 3 rows



In [104]:
df.groupBy('user_id').agg(F.sum(df['quantity']*df['price']).alias('Total Gasto')).orderBy(F.desc('Total Gasto')).show()

+-------+-----------+
|user_id|Total Gasto|
+-------+-----------+
|     U1|      340.0|
|     U3|      145.0|
|     U5|      115.0|
|     U4|      106.0|
|     U2|      105.0|
|     U7|      105.0|
|     U8|      100.0|
|     U9|      100.0|
|     U6|       99.0|
|    U10|       90.0|
+-------+-----------+



In [112]:
df.groupBy('order_id').agg(F.sum('quantity').alias('quantidade')).agg(F.avg('quantidade').alias('Média itens')).show()

+-----------+
|Média itens|
+-----------+
|       3.65|
+-----------+

